In [39]:
# Стандартные библиотеки
import pandas as pd
import requests

# Сторонние библиотеки
import datetime
from bs4 import BeautifulSoup
import html5lib

PATH_DIR = r'D:\prj_anal\housing-prices-mortgage-rates' 

TODO Содержание

## 1. Данные по ипотечным ставкам

Источник: ЦБ РФ (https://cbr.ru/)  
Период: 2018–2025

- Ключевая ставка ЦБ РФ (%)
- Средняя ипотечная ставка (%)
- Тип ипотеки (например, "новостройки", "вторичное жилье")

In [103]:
start_date =  datetime.date(2018, 1, 1).strftime('%d.%m.%Y')
end_date =  datetime.date(2024, 12, 31).strftime('%d.%m.%Y')

# Ключевая ставка 
url_key_rate = f"https://cbr.ru/hd_base/KeyRate/?UniDbQuery.Posted=True&UniDbQuery.From={start_date}&UniDbQuery.To={end_date}"

response = requests.get(url_key_rate)
response.raise_for_status()

key_rate_df = pd.read_html(url_key_rate)[0]

key_rate_df.columns = ["date", "key_rate"]
key_rate_df["date"] = pd.to_datetime(key_rate_df["date"], format="%d.%m.%Y")
key_rate_df["key_rate"] = key_rate_df["key_rate"]/100

key_rate_df.info()
print(key_rate_df)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1761 entries, 0 to 1760
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      1761 non-null   datetime64[ns]
 1   key_rate  1761 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 27.6 KB
           date  key_rate
0    2024-12-30     21.00
1    2024-12-28     21.00
2    2024-12-27     21.00
3    2024-12-26     21.00
4    2024-12-25     21.00
5    2024-12-24     21.00
6    2024-12-23     21.00
7    2024-12-20     21.00
8    2024-12-19     21.00
9    2024-12-18     21.00
10   2024-12-17     21.00
11   2024-12-16     21.00
12   2024-12-13     21.00
13   2024-12-12     21.00
14   2024-12-11     21.00
15   2024-12-10     21.00
16   2024-12-09     21.00
17   2024-12-06     21.00
18   2024-12-05     21.00
19   2024-12-04     21.00
20   2024-12-03     21.00
21   2024-12-02     21.00
22   2024-11-29     21.00
23   2024-11-28     21.00


In [104]:
# Ипотечные жилищные кредиты, предоставленные физическим лицам-резидентам,
# и приобретенные права требования по ипотечным жилищным кредитам

xlsx_file_path = r"..\data\02_02_Mortgage.xlsx"

mortgage_df = pd.read_excel(xlsx_file_path, sheet_name='в рублях', skiprows=1)
mortgage_df = mortgage_df.set_index('Отчетная дата').T
mortgage_df.columns = [
    'loans_issued_monthly_count',
    'loans_issued_monthly_volume_millions_rub',
    'total_debt_millions_rub',
    'overdue_debt_millions_rub',
    'acquired_mortgage_rights_debt_millions_rub',
    'total_debt_including_acquired_rights_millions_rub',
    'weighted_average_loan_term_months',
  'weighted_average_interest_rate_percent'
]

# TODO Добавим новый признак "Тип ипотеки" и разбиение "новостройка = 1", "вторичное жилье = 0"
mortgage_df['Mortgage Type'] = 0

mortgage_df.info()
print(mortgage_df)


<class 'pandas.core.frame.DataFrame'>
Index: 85 entries, 01.01.2018 to 01.01.2025
Data columns (total 9 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   loans_issued_monthly_count                         85 non-null     float64
 1   loans_issued_monthly_volume_millions_rub           85 non-null     float64
 2   total_debt_millions_rub                            85 non-null     float64
 3   overdue_debt_millions_rub                          85 non-null     float64
 4   acquired_mortgage_rights_debt_millions_rub         85 non-null     float64
 5   total_debt_including_acquired_rights_millions_rub  85 non-null     float64
 6   weighted_average_loan_term_months                  85 non-null     float64
 7   weighted_average_interest_rate_percent             85 non-null     float64
 8   Mortgage Type                                      85 non-null     int64  
dtypes

In [105]:
# Ипотечные жилищные кредиты, предоставленные физическим лицам-резидентам 
# под залог прав требования по договорам участия в долевом строительстве (ДДУ), 
# и приобретенные права требования по ипотечным жилищным кредитам под залог ДДУ

xlsx_file_path = r"..\data\02_03_Scpa_mortgage.xlsx"

scpa_mortgage_df = pd.read_excel(xlsx_file_path, sheet_name='в рублях', skiprows=[0, 10])
scpa_mortgage_df = scpa_mortgage_df.set_index('Отчетная дата').T
scpa_mortgage_df.columns = [
    'loans_issued_monthly_count',
    'loans_issued_monthly_volume_millions_rub',
    'total_debt_millions_rub',
    'overdue_debt_millions_rub',
    'acquired_mortgage_rights_debt_millions_rub',
    'total_debt_including_acquired_rights_millions_rub',
    'weighted_average_loan_term_months',
    'weighted_average_interest_rate_percent'
]

# TODO Добавим новый признак "Тип ипотеки" и разбиение "новостройка = 1", "вторичное жилье = 0"
scpa_mortgage_df['Mortgage Type'] = 1

scpa_mortgage_df.info()
print(scpa_mortgage_df)

<class 'pandas.core.frame.DataFrame'>
Index: 85 entries, 01.01.2018 to 01.01.2025
Data columns (total 9 columns):
 #   Column                                             Non-Null Count  Dtype 
---  ------                                             --------------  ----- 
 0   loans_issued_monthly_count                         85 non-null     object
 1   loans_issued_monthly_volume_millions_rub           85 non-null     object
 2   total_debt_millions_rub                            85 non-null     object
 3   overdue_debt_millions_rub                          85 non-null     object
 4   acquired_mortgage_rights_debt_millions_rub         85 non-null     object
 5   total_debt_including_acquired_rights_millions_rub  85 non-null     object
 6   weighted_average_loan_term_months                  85 non-null     object
 7   weighted_average_interest_rate_percent             85 non-null     object
 8   Mortgage Type                                      85 non-null     int64 
dtypes: int64(1),

### Описание таблицы ипотечной ставки "mortgage_df" и "scpa_mortgage_df"

__ИЖК (Ипотечное жилищное кредитование)__  

__ИЖК по ДДУ (Ипотечное жилищное кредитование по договорам долевого участия)__   

| Имя столбца в таблице                                      | Описание                                                                 |
|------------------------------------------------------------|--------------------------------------------------------------------------|
| `loans_issued_monthly_count`                               | Количество предоставленных кредитов за месяц, единиц                    |
| `loans_issued_monthly_volume_millions_rub`                 | Объем предоставленных кредитов за месяц, млн руб.                       |
| `total_debt_millions_rub`                                  | Задолженность по предоставленным кредитам, млн руб., в том числе        |
| `overdue_debt_millions_rub`                                | Просроченная задолженность по предоставленным кредитам, млн руб.        |
| `acquired_mortgage_rights_debt_millions_rub`               | Задолженность по приобретенным правам требования по ипотечным кредитам, млн руб. |
| `total_debt_including_acquired_rights_millions_rub`        | Задолженность по предоставленным кредитам с учетом приобретенных прав требования, млн руб. |
| `weighted_average_loan_term_months`                        | Средневзвешенный срок кредитования по кредитам, выданным в течение месяца, месяцев |
| `weighted_average_interest_rate_percent`                   | Средневзвешенная ставка по кредитам, выданным в течение месяца, %       |


In [ ]:
# TODO: Ипотечная ставка по регионам

In [ ]:
# TODO: Формирование единого dataframe

## 2. Данные по ценам на недвижимость

Источник: Росстат, Kaggle, порталы недвижимости (ЦИАН, Avito)   
Период: 2018-2025

- Тип недвижимости (например, "квартира", "дом")
- Средняя цена за квадратный метр (руб)
- Средняя общая стоимость объекта (руб)
- Количество предложений

## 3. Макроэкономические данные

Источник: Росстат, ЦБ РФ  
Период: 2018-2025

- Уровень инфляции (%)
- Средний доход населения (руб)
- Курс доллара (руб)
- Курс евро (руб)
